In [ ]:
## FUNCTION 1
def search_companies_named(name, simple=False, save=False, df=True):

    '''

    Takes in a company name or part of it (string) and returns a dataframe with information about all companies named like that.

    Parameters:
    name (str): name of the company (or part of it) to search for. Not caps sensitive.
    simple (bool): False by default. If True, returns a simplified version of the dictionary with only the name, ehraid, and legalSeat of the company.
    save (bool): False by default. If True, saves the data to a JSON file called "name_companies.json". If simple is also True at the same time, saves the data to a JSON file called "name_companies_simple.json" instead.
    df (bool): True by default. If True, returns the data as a pandas DataFrame. If False, returns the data as a list of dictionaries.

    Returns:
    
    if df=False: 
    list_of_companies (list): a list of dictionaries, each containing information about a company found in the search

    if df=True (default): 
    list_of_companies (DataFrame): a pandas DataFrame containing information about all companies found in the search

    The dataframe in the list will contain the following information about the company:

    'name':                 <-- name of the company,
    'ehraid':               <-- unique identifier of the company,
    'legalSeat':            <-- location of the company's legal seat,
    'status':               <-- status of the company (active or not),
    'deleteDate':           <-- date when the company was deleted (if applicable),
    'wasTakenOverBy':       <-- name of the company that took over this one (if applicable),
    'purpose':              <-- purpose of the company,
    'legalForm':            <-- legal form of the company,
    'cancelled':            <-- boolean value indicating if the company is cancelled or not,
    'n_of_shabPub':         <-- number of publications in the Swiss Official Gazette of Commerce,
    'n_of_branchOffices':   <-- number of branch offices of the company,
    'n_of_oldNames':        <-- number of old names of the company,
    'n_of_hasTakenOver':    <-- number of companies taken over by this one,
    'town':                 <-- town where the company is located,
    'swissZipCode':         <-- Swiss zip code of the company,

    if simple=True, the dataframe/list of dictionaries will only contain the following:

    'name':                 <-- name of the company,
    'ehraid':               <-- unique identifier of the company,
    'legalSeat':            <-- location of the company's legal seat,

    '''
    # check if input is a string
    if type(name) != str:
        return "Please provide a string of the name of the company (or part of it) as input. Example: 'UBS' returns information about all companies with 'UBS' in their name."
    
    # access the information
    search_url = 'https://www.zefix.ch/ZefixREST/api/v1/firm/search.json'
    search_query = {
        "name" : name,
        "searchType" : "exact"
    }
    headers = {'Content-Type': 'application/json'}
    json_data = json.dumps(search_query)
    r = requests.post(search_url, headers=headers, data=json_data)

    # check if the request was successful, return error message if not
    if r.status_code == 400:
        return "400 Bad Request Error"
    elif r.status_code == 401:
        return "401 Unauthorized Error"
    elif r.status_code == 403:
        return "403 Forbidden Error"
    elif r.status_code == 404:
        return "404 Not Found Error"
    elif r.status_code == 408:
        return "408 Request Timeout Error"
    elif r.status_code == 500:
        return "500 Internal Server Error"
    elif r.status_code == 502:
        return "502 Bad Gateway Error"
    elif r.status_code == 504:
        return "504 Gateway Timeout Error"
    elif r.status_code != 200:
        return "Unexpected Error, the request was not successful"

    # convert response to json
    data = r.json()

    # create an empty list to store the information of the companies found in the search
    firm_data = []

    # populate the firm_data list with the information of the companies found in the search
    for i in range(len(data['list'])):
        firm_data.append(data['list'][i])

    # create an empty list to store the results for each of the companies found in the search
    list_of_companies = []

    # if option simple is true, return the simplified version of the dictionary
    if simple==True:
        for n in range(len(firm_data)):
            company = {}

            company['name'] = firm_data[n]['name']

            company['ehraid'] = firm_data[n]['ehraid']

            company['legalSeat'] = firm_data[n]['legalSeat']

            company['cantonalExcerptWeb'] = firm_data[n]['cantonalExcerptWeb']

            list_of_companies.append(company)

        # Save data to JSON file if option save is True
        if simple == True and save == True:
            with open(f'{name}_companies_simple.json', 'w') as file:
                json.dump(list_of_companies, file, indent=4)
            print(f"Data saved to {name}_companies_simple.json")
        
        # dataframe option
        if df==True:
            list_of_companies = pd.DataFrame(list_of_companies)
        
        return list_of_companies
    
    for n in range(len(firm_data)):

        # select ehraid
        ehraid = firm_data[n]['ehraid']

        # get information about the firm using its ehraid
        company_url = f"https://www.zefix.ch/ZefixREST/api/v1/firm/{ehraid}"
        r = requests.get(company_url)
        d = r.json()

        url = "https://www.zefix.ch/ZefixREST/api/v1/legalForm"
        response = requests.get(url)
        legal_forms = response.json()

        # create a dictionary with legal form id as key
        legal_forms_dictionary = {item['id']: item for item in legal_forms}

        # create empty dictionary to store the searched company information
        company={}
        
        # attributes to keep
        attributes = ['name', 'ehraid', 'legalSeat', 'cantonalExcerptWeb', 'status', 'deleteDate', 'wasTakenOverBy', 'purpose']

        # get request from zefix to get the desired company information
        company_url = f"https://www.zefix.ch/ZefixREST/api/v1/firm/{ehraid}"
        r = requests.get(company_url)
        data = r.json()
    
        for attrib in attributes:
            company[attrib] = data[attrib]
        
        #translate the information in 'purpuse' to english
        if company["purpose"]:
            translator = Translator()
            purpose_english = translator.translate(company['purpose'], dest='en')
            company['purpose'] = purpose_english.text

        #translate the information in 'status' to english
        if company["status"]:
            if company['status'].upper() == 'EXISTIEREND':
                company['status'] = 'Active'
            elif company['status'].upper() == 'GELOESCHT':
                company['status'] = 'Deleted'
            elif company['status'].upper() == 'IN_AUFLOESUNG':
                company['status'] = 'In Liquidation'
            else:
                translator = Translator()
                status_english = translator.translate(company['status'], dest='en')
                company['status'] = status_english.text

        # get the English name of a legal form associated with the given ID provided by the API
        firm_legal_form_name = legal_forms_dictionary[data['legalFormId']]['name']['en']

        # fill the legalForm field in the dictionary with the english name we just found
        company['legalForm'] = firm_legal_form_name 
        
        # cancelled field      
        if data['deleteDate'] is not None:
            company['cancelled'] = True
        else:
            company['cancelled'] = False
        
        # defining all the 'n_...' attributes by counting 
        len_fields = [ 'shabPub', 'branchOffices', 'oldNames', 'hasTakenOver']
        for field in len_fields:
            if data[field] is not None:
                company[f"n_of_{field}"] = len(data[field])
            else:
                company[f"n_of_{field}"] = 0
        
        # town/swissZipCode fields
        company['town'] = data['address']['town']
        company['swissZipCode'] = data['address']['swissZipCode']

        list_of_companies.append(company)

    # Save data to JSON file if option save is True
    if save == True:
        with open(f'{name}_companies.json', 'w') as file:
            json.dump(list_of_companies, file, indent=4)
        print(f"Data saved to {name}_companies.json")        

    # dataframe option
    if df==True:
        list_of_companies = pd.DataFrame(list_of_companies)

    return list_of_companies

In [ ]:
##EXAMPLE FUNCTION 1
search_companies_named('UBS', save=True)
search_companies_named('LALIVE', simple=True)
search_companies_named('MSC')
search_companies_named('UBS', save=False, df=True)